In [86]:
import pandas as pd

In [87]:
# using the query in get_listings.sql to generate data
df = pd.read_csv('data/cincy_listing_history_2023_08_22 (2).csv')
df.head()

,status,listing_id,cancellation_date,close_date,close_price,expiration_date,list_price,listing_contract_date,original_list_price,previous_list_price,price_change_timestamp,purchase_contract_date,withdrawn_date
0,Closed,1001166,08/08/06,NaN,NaN,28/10/06,164900.0,02/05/06,179900.0,NaN,19/11/22 17:23,08/08/06,NaN
1,Closed,1001652,NaN,04/08/06,137750.0,04/09/06,NaN,04/05/06,NaN,NaN,NaN,03/06/06,NaN
2,Closed,1001800,22/06/06,NaN,NaN,04/11/06,249500.0,04/05/06,249900.0,NaN,19/11/22 17:24,22/06/06,NaN
3,Closed,1001971,09/04/07,NaN,NaN,11/05/07,110000.0,04/05/06,132500.0,NaN,19/11/22 17:25,09/04/07,NaN
4,Closed,1002140,NaN,24/07/06,25725.0,05/08/06,NaN,05/05/06,35000.0,NaN,NaN,24/06/06,NaN


In [88]:
for c in df.columns:
    if '_date' in c:
        df[f'{c}'] = pd.to_datetime(df[f'{c}'], dayfirst=True, errors='coerce')
    elif '_price' in c:
        df[f'{c}'] = pd.to_numeric(df[f'{c}'], errors='coerce')
        df[f'{c}'] = [1 if not pd.isna(x) else 0 for x in df[f'{c}']]
    elif '_timestamp' in c:
        df[f'{c}'] = pd.to_datetime(df[f'{c}'], errors='coerce')

In [89]:
def flatten(row):
    tmp = {'listing_id': [], 'state': [], 'time': []}
    for c in df.columns:
        if '_date' in c or '_timestamp' in c:
            tmp['listing_id'].append(row['listing_id'])
            tmp['state'].append(c)
            tmp['time'].append(row[c])
    return pd.DataFrame(tmp)

In [90]:
df_flat_list = [flatten(row) for index, row in df.iterrows()]

In [91]:
# df_flat = pd.concat(df_flat_list)
df_flat_list = [x.sort_values('time') for x in df_flat_list]

In [92]:
# df_flat.head()

In [93]:
# df_flat.shape

In [94]:
# df_flat.sort_values(by=['listing_id', 'time'], inplace=True)

In [95]:
# df_flat

In [96]:
df_collapsed = {'listing_id': [], 'key': []}
for d in df_flat_list:
    ret_val = ''
    listing_id = None
    for i in d[~d['time'].isna()]['state']:
        ret_val += i
        ret_val += '|'
    listing_id = d['listing_id'].values[0]
    df_collapsed['listing_id'].append(listing_id)
    df_collapsed['key'].append(ret_val)
df_collapsed = pd.DataFrame(df_collapsed)
df_collapsed.head()

,listing_id,key
0,1001166,listing_contract_date|cancellation_date|purcha...
1,1001652,listing_contract_date|purchase_contract_date|c...
2,1001800,listing_contract_date|cancellation_date|purcha...
3,1001971,listing_contract_date|cancellation_date|purcha...
4,1002140,listing_contract_date|purchase_contract_date|c...


In [97]:
crmls_events_order = df_collapsed.groupby('key')['listing_id'].nunique().sort_values(ascending=False)
print(crmls_events_order.head())
crmls_events_order.to_csv('data/crmls_events_order.csv')

key
listing_contract_date|cancellation_date|purchase_contract_date|expiration_date|price_change_timestamp|    1409
listing_contract_date|purchase_contract_date|cancellation_date|expiration_date|price_change_timestamp|    1104
listing_contract_date|purchase_contract_date|close_date|expiration_date|                                  1008
listing_contract_date|expiration_date|                                                                     554
listing_contract_date|cancellation_date|expiration_date|price_change_timestamp|                            549
Name: listing_id, dtype: int64


In [98]:
# df_flat.to_csv('data/crmls_events_sorted.csv')
df_collapsed.to_csv('data/crmls_events_collapsed.csv')

In [99]:
print(df.shape, df['listing_id'].nunique())

(8584, 13) 8562


In [100]:
p_keys = []
for index, row in df.iterrows():
    price_keys = []
    for c in df.columns:
        if '_date' not in c and c != 'listing_id' and 'status' not in c and '_timestamp' not in c:
            if not pd.isna(row[f'{c}']) and row[f'{c}'] == 1:
                price_keys.append(c)
    price_keys.sort()
    price_key = ''
    for p in price_keys:
        price_key += p
        price_key += '|'
    p_keys.append(price_key)
df['price_key'] = p_keys
df.head()

,status,listing_id,cancellation_date,close_date,close_price,expiration_date,list_price,listing_contract_date,original_list_price,previous_list_price,price_change_timestamp,purchase_contract_date,withdrawn_date,price_key
0,Closed,1001166,2006-08-08,NaT,0,2006-10-28,1,2006-05-02,1,0,2022-11-19 17:23:00,2006-08-08,NaT,list_price|original_list_price|
1,Closed,1001652,NaT,2006-08-04,1,2006-09-04,0,2006-05-04,0,0,NaT,2006-06-03,NaT,close_price|
2,Closed,1001800,2006-06-22,NaT,0,2006-11-04,1,2006-05-04,1,0,2022-11-19 17:24:00,2006-06-22,NaT,list_price|original_list_price|
3,Closed,1001971,2007-04-09,NaT,0,2007-05-11,1,2006-05-04,1,0,2022-11-19 17:25:00,2007-04-09,NaT,list_price|original_list_price|
4,Closed,1002140,NaT,2006-07-24,1,2006-08-05,0,2006-05-05,1,0,NaT,2006-06-24,NaT,close_price|original_list_price|


In [101]:
df_collapsed = df_collapsed.merge(df.drop(columns=[c for c in df.columns if ('date') in c or ('timestamp') in c]), how='left', on='listing_id')
df_collapsed

,listing_id,key,status,close_price,list_price,original_list_price,previous_list_price,price_key
0,1001166,listing_contract_date|cancellation_date|purcha...,Closed,0,1,1,0,list_price|original_list_price|
1,1001652,listing_contract_date|purchase_contract_date|c...,Closed,1,0,0,0,close_price|
2,1001800,listing_contract_date|cancellation_date|purcha...,Closed,0,1,1,0,list_price|original_list_price|
3,1001971,listing_contract_date|cancellation_date|purcha...,Closed,0,1,1,0,list_price|original_list_price|
4,1002140,listing_contract_date|purchase_contract_date|c...,Closed,1,0,1,0,close_price|original_list_price|
...,...,...,...,...,...,...,...,...
8625,999091,listing_contract_date|purchase_contract_date|c...,Closed,0,1,1,0,list_price|original_list_price|
8626,999850,listing_contract_date|purchase_contract_date|c...,Closed,1,0,0,0,close_price|
8627,999856,listing_contract_date|expiration_date|price_ch...,Expired,0,1,0,0,list_price|
8628,999927,listing_contract_date|purchase_contract_date|c...,Closed,1,0,0,0,close_price|


In [102]:
df_collapsed.to_csv('data/cincy_listing_history_out.csv')

In [103]:
df[df['listing_id'] == 1005000]

,status,listing_id,cancellation_date,close_date,close_price,expiration_date,list_price,listing_contract_date,original_list_price,previous_list_price,price_change_timestamp,purchase_contract_date,withdrawn_date,price_key
14,Closed,1005000,2007-02-15,NaT,0,2007-05-18,1,2006-05-18,1,0,2022-11-19 17:27:00,2007-02-14,NaT,list_price|original_list_price|
